In [1]:
import warnings

import numpy as np
import pandas as pd
import useful_rid_code as rid
from openpyxl import load_workbook

warnings.simplefilter("ignore")

# RID 

## Fill-in OSPAR reporting template (parameterised)

This notebook is "parameterised" for use with Papermill. The cell below has the tag `parameters`, which means the entire notebook can be called from `01_recalculate_ospar_1990-2016_main.ipynb`.

In [2]:
# This cell is tagged 'parameters' for use with Papermill
# https://papermill.readthedocs.io/en/latest/index.html
year = 1990
user = ""
pw = ""

In [3]:
# Parameters
user = "jes"
pw = "BeakabusNov21.."
year = 1997


## 1. Get summary data

The notebooks `summary_table_{year}.ipynb` calculate summary values for Table 3. Rather than reading these values from Word, it is easier to repeat the code to summarise the raw data again. This is the data that needs writing to the OSPAR template.

### 1.1. Monitored areas

In [4]:
# Read data
in_csv = f"../../../Results/Loads_CSVs/loads_and_flows_all_sites_{year}.csv"
mon_df = pd.read_csv(in_csv)
del mon_df["new_rid_group"]

# Group by OSPAR region
mon_df1 = mon_df.groupby(["ospar_region", "old_rid_group"]).sum()

# Totals for Norway
mon_df2 = mon_df.groupby("old_rid_group").sum().reset_index()
mon_df2["ospar_region"] = "NORWAY"
mon_df2.set_index(["ospar_region", "old_rid_group"], inplace=True)

# Combine
mon_df = pd.concat([mon_df1, mon_df2], axis=0)

# Cols of interest
cols = [i for i in mon_df.columns if i.split("_")[1] != "Est"]
mon_df = mon_df[cols]
del mon_df["station_id"], mon_df["mean_q_1000m3/day"]

# Convert units
mon_df["Hg_kg"] = mon_df["Hg_kg"] / 1000.0  # kg to tonnes
mon_df["NH4-N_tonnes"] = mon_df["NH4-N_tonnes"] / 1000.0  # tonnes to ktonnes
mon_df["NO3-N_tonnes"] = mon_df["NO3-N_tonnes"] / 1000.0  # tonnes to ktonnes
mon_df["TOTN_tonnes"] = mon_df["TOTN_tonnes"] / 1000.0  # tonnes to ktonnes
mon_df["TOTP_tonnes"] = mon_df["TOTP_tonnes"] / 1000.0  # tonnes to ktonnes
mon_df["PO4-P_tonnes"] = mon_df["PO4-P_tonnes"] / 1000.0  # tonnes to ktonnes
mon_df["SPM_tonnes"] = mon_df["SPM_tonnes"] / 1000.0  # tonnes to ktonnes

# Units are correct, so remove
mon_df.columns = [i.split("_")[0] for i in mon_df.columns]

mon_df.round(0)

As   Cd    Cr     Cu   Hg  NH4-N  NO3-N  \
ospar_region        old_rid_group                                              
LOFOTEN-BARENTS SEA rid_108         1.0  0.0   3.0    6.0  0.0    0.0    0.0   
                    rid_11          2.0  0.0   0.0    3.0  0.0    0.0    0.0   
                    rid_36          2.0  0.0   9.0   34.0  0.0    0.0    1.0   
NORTH SEA           rid_108         1.0  0.0   0.0   11.0  0.0    0.0    3.0   
                    rid_11          2.0  0.0   0.0    2.0  0.0    0.0    0.0   
                    rid_36          1.0  0.0   0.0    9.0  0.0    0.0    4.0   
NORWEGIAN SEA2      rid_108         2.0  0.0   6.0   21.0  0.0    0.0    1.0   
                    rid_11          1.0  0.0   1.0   41.0  0.0    0.0    1.0   
                    rid_36          6.0  0.0  14.0   35.0  0.0    0.0    3.0   
SKAGERAK            rid_108         1.0  0.0   2.0    3.0  0.0    0.0    1.0   
                    rid_11          4.0  1.0   1.0   71.0  0.0    1.0   14.0   
                    rid_36          2.0  0.0   0.0    4.0  0.0    0.0    1.0   
NORWAY              rid_108         5.0  1.0  11.0   40.0  0.0    0.0    6.0   
                    rid_11          8.0  1.0   2.0  117.0  0.0    1.0   15.0   
                    rid_36         11.0  1.0  22.0   81.0  0.0    1.0    9.0   

                                     Ni  PO4-P    Pb    SPM      SiO2  \
ospar_region        old_rid_group                                       
LOFOTEN-BARENTS SEA rid_108         4.0    0.0   1.0    8.0   25537.0   
                    rid_11          1.0    0.0   0.0    3.0   11017.0   
                    rid_36         40.0    0.0   5.0   25.0   72679.0   
NORTH SEA           rid_108         6.0    0.0   4.0   23.0   25527.0   
                    rid_11          1.0    0.0   0.0    5.0    3421.0   
                    rid_36          2.0    0.0   5.0   24.0   26982.0   
NORWEGIAN SEA2      rid_108         5.0    0.0   2.0   28.0   23150.0   
                    rid_11          6.0    0.0   4.0   33.0   15625.0   
                    rid_36         26.0    0.0   6.0  235.0   60765.0   
SKAGERAK            rid_108         3.0    0.0   1.0   11.0    8989.0   
                    rid_11         30.0    0.0  14.0  263.0  106654.0   
                    rid_36          3.0    0.0   3.0   10.0   14479.0   
NORWAY              rid_108        18.0    0.0   8.0   71.0   83203.0   
                    rid_11         38.0    0.0  18.0  304.0  136717.0   
                    rid_36         71.0    0.0  20.0  294.0  174904.0   

                                        TOC  TOTN  TOTP     Zn   Ag  
ospar_region        old_rid_group                                    
LOFOTEN-BARENTS SEA rid_108         20143.0   1.0   0.0    7.0  0.0  
                    rid_11           9405.0   0.0   0.0    1.0  0.0  
                    rid_36          47071.0   3.0   0.0   64.0  0.0  
NORTH SEA           rid_108         21138.0   5.0   0.0   52.0  0.0  
                    rid_11           3429.0   1.0   0.0   27.0  0.0  
                    rid_36          40214.0   7.0   0.0   80.0  0.0  
NORWEGIAN SEA2      rid_108         24025.0   3.0   0.0   30.0  0.0  
                    rid_11          33618.0   2.0   0.0  108.0  0.0  
                    rid_36          88657.0   8.0   0.0   80.0  0.0  
SKAGERAK            rid_108         10924.0   2.0   0.0   25.0  0.0  
                    rid_11         175194.0  21.0   0.0  213.0  0.0  
                    rid_36          26450.0   3.0   0.0   39.0  0.0  
NORWAY              rid_108         76231.0  11.0   0.0  115.0  0.0  
                    rid_11         221647.0  25.0   1.0  348.0  0.0  
                    rid_36         202391.0  20.0   1.0  263.0  0.0

### 1.2. Unmonitored areas

In [5]:
# Read data
in_csv = f"../../../Results/Unmon_loads/teotil2_ospar_unmonitored_loads_{year}.csv"
umon_df = pd.read_csv(in_csv, index_col=0)

# Rename cols
umon_df.columns = [i.replace("RENSEANLEGG", "sew") for i in umon_df.columns]
umon_df.columns = [i.replace("INDUSTRI", "ind") for i in umon_df.columns]
umon_df.columns = [i.replace("_tonn", "") for i in umon_df.columns]
umon_df.columns = [i.replace("AQUAKULTUR", "fish") for i in umon_df.columns]

# Add missing columns if necessary
cols = ["sew_Hg", "sew_S.P.M.", "ind_Hg"]
for col in cols:
    if col not in umon_df.columns:
        umon_df[col] = 0

# Convert Hg to kgs
umon_df["sew_Hg"] = umon_df["sew_Hg"] * 1000
umon_df["ind_Hg"] = umon_df["ind_Hg"] * 1000

umon_df.round(0)

,flow,sew_n,sew_p,ind_n,ind_p,fish_n,fish_p,diff_n,diff_p,sew_po4,...,ind_Pb,ind_Cd,ind_Cu,ind_Zn,ind_Ni,ind_Cr,ind_Hg,fish_Cu,sew_Hg,sew_S.P.M.
ospar_region,,,,,,,,,,,,,,,,,,,,,
NORWAY,442881.0,13899.0,1161.0,3263.0,194.0,17063.0,3587.0,32172.0,649.0,697.0,...,4.0,1.0,9.0,45.0,9.0,0.0,21.0,309.0,0,0
LOFOTEN-BARENTS SEA,138608.0,1602.0,197.0,1.0,0.0,2524.0,532.0,6009.0,110.0,118.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,46.0,0,0
NORTH SEA,121274.0,3347.0,316.0,399.0,48.0,6078.0,1282.0,11473.0,189.0,190.0,...,4.0,1.0,1.0,44.0,5.0,0.0,7.0,110.0,0,0
NORWEGIAN SEA2,175238.0,3490.0,420.0,1461.0,87.0,8431.0,1766.0,11798.0,258.0,252.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,153.0,0,0
SKAGERAK,7761.0,5460.0,228.0,1403.0,59.0,31.0,6.0,2892.0,93.0,137.0,...,0.0,0.0,8.0,1.0,3.0,0.0,10.0,1.0,0,0


### 1.3. Loads for 11 main rivers

In [6]:
# Read data
in_csv = f"../../../Results/Loads_CSVs/loads_and_flows_all_sites_{year}.csv"
rid11_df = pd.read_csv(in_csv, index_col=0)
del rid11_df["new_rid_group"]

# Get data for RID11
rid11_df = rid11_df.query('old_rid_group == "rid_11"')

# Tidy
del rid11_df["station_code"], rid11_df["station_name"]
del rid11_df["old_rid_group"], rid11_df["ospar_region"]
del rid11_df["mean_q_1000m3/day"]

cols = [i for i in rid11_df.columns if i.split("_")[1] != "Est"]
rid11_df = rid11_df[cols]

# Convert units
rid11_df["Hg_kg"] = rid11_df["Hg_kg"] / 1000.0  # kg to tonnes
rid11_df["NH4-N_tonnes"] = rid11_df["NH4-N_tonnes"] / 1000.0  # tonnes to ktonnes
rid11_df["NO3-N_tonnes"] = rid11_df["NO3-N_tonnes"] / 1000.0  # tonnes to ktonnes
rid11_df["TOTN_tonnes"] = rid11_df["TOTN_tonnes"] / 1000.0  # tonnes to ktonnes
rid11_df["TOTP_tonnes"] = rid11_df["TOTP_tonnes"] / 1000.0  # tonnes to ktonnes
rid11_df["PO4-P_tonnes"] = rid11_df["PO4-P_tonnes"] / 1000.0  # tonnes to ktonnes
rid11_df["SPM_tonnes"] = rid11_df["SPM_tonnes"] / 1000.0  # tonnes to ktonnes

# Tidy cols
rid11_df.columns = [i.split("_")[0] for i in rid11_df.columns]

rid11_df.head()

,As,Cd,Cr,Cu,Hg,NH4-N,NO3-N,Ni,PO4-P,Pb,SPM,SiO2,TOC,TOTN,TOTP,Zn,Ag
station_id,,,,,,,,,,,,,,,,,
29612,0.684406,0.111882,0.0,7.603823,0.001195,0.131389,2.330379,4.530984,0.013284,1.007258,12.969660,19318.240048,27957.318598,3.646984,0.037899,23.925230,0.008522
29779,1.504786,0.008637,0.0,2.561425,0.000000,0.004038,0.113860,0.828238,0.014407,0.107659,2.910295,11017.182633,9404.912003,0.399045,0.025312,1.178118,0.004120
29821,1.553845,0.028775,0.0,2.014244,0.002877,0.043162,0.235954,0.575498,0.002877,0.028775,2.992591,2963.815627,2618.516719,0.546723,0.011510,26.185167,0.007964
29782,0.394855,0.110721,0.0,13.320764,0.003323,0.056416,0.355076,3.033072,0.019584,3.617206,25.760906,8449.036157,22747.405039,1.463744,0.035854,40.555946,0.008460
36225,0.064008,0.006957,0.0,0.556590,0.000278,0.021568,0.195502,0.000000,0.007514,0.098795,0.470319,890.544526,751.396944,0.310299,0.023377,1.252328,0.001402


## 2. Fill-in template

The template is usually sent each year by Csilla at NIBIO. However, it doesn't seem to change, so old versions can also be used e.g. here:

    ../../../Results/OSPAR/Blank_Template

In [7]:
# Copy of template to update
temp_path = rid.copy_ospar_template(year)

In [8]:
def update_spreadsheet_point_sources(xlsx, sheet, pars, src, df):
    """Update the OSPAR template for point source data.

    Args:
        xslx:  Str. Path to Excel template
        sheet: Str. Sheet name to update
        pars:  List. Parameter names in template to fill-in
        src:   Str. Type of input ('sew', 'ind', 'fish')
        df:    Dataframe. Values to fill-in

    Returns:
        None. The template is updated and saved.
    """
    import pandas as pd
    from openpyxl import load_workbook

    # Map Excel headings to df cols
    par_dict = {
        "SPM": "S.P.M.",
        "TOC": "TOC",
        "PO4-P": "po4",
        "P-Total": "p",
        "NO3-N": "no3",
        "NH4-N": "nh4",
        "N-Total": "n",
        "As": "As",
        "Pb": "Pb",
        "Cd": "Cd",
        "Cu": "Cu",
        "Zn": "Zn",
        "Ni": "Ni",
        "Total Cr": "Cr",
        "Hg": "Hg",
    }

    # Map template names to df names
    names_dict = {
        "Norwegian Sea (NO)": "NORWEGIAN SEA2",
        "Barents Sea (NO)": "LOFOTEN-BARENTS SEA",
        "Skagerrak (NO)": "SKAGERAK",
        "North Sea (NO)": "NORTH SEA",
        "Norway Total": "NORWAY",
    }

    # Open new file and get sheet
    wb = load_workbook(filename=xlsx)
    ws = wb[sheet]

    # Get row numbers
    row_dict = {}
    for item in ws["B12" : "B%s" % ws.max_row]:
        # Get cell properties
        cell = item[0]
        name = cell.value
        row = cell.row
        row_dict[name] = row

    # Get col numbers
    col_dict = {}
    for cell in ws["E9":"AK9"][0]:
        # Get cell properties
        par = cell.value
        col = cell.column
        col_dict[par] = col

    # Update spreadsheet
    for reg in names_dict.keys():
        for par in pars:
            # Get value from df
            try:
                val = df.loc[names_dict[reg], "%s_%s" % (src, par_dict[par])]
            except KeyError:
                val = 0

            # Get cell co-ords
            row = row_dict[reg] + 2
            col = col_dict[par]

            # Write value
            ws.cell(row=row, column=col).value = val

    # Save
    wb.save(xlsx)

### 2.1. Sheet 5a: Sewage effluents

Note these are the values from Table 3 for **unmonitored areas** and not the total sewage inputs for each of the OSPAR areas. This is the same as what Tore reported previously.

In [9]:
# Get Sewage data
cols = [i for i in umon_df.columns if i.split("_")[0] == "sew"]
sew_df = umon_df[cols].copy()

# Convert units
sew_df["sew_Hg"] = sew_df["sew_Hg"] / 1000.0  # kg to tonnes
sew_df["sew_nh4"] = sew_df["sew_nh4"] / 1000.0  # tonnes to ktonnes
sew_df["sew_no3"] = sew_df["sew_no3"] / 1000.0  # tonnes to ktonnes
sew_df["sew_n"] = sew_df["sew_n"] / 1000.0  # tonnes to ktonnes
sew_df["sew_po4"] = sew_df["sew_po4"] / 1000.0  # tonnes to ktonnes
sew_df["sew_p"] = sew_df["sew_p"] / 1000.0  # tonnes to ktonnes
sew_df["sew_S.P.M."] = sew_df["sew_S.P.M."] / 1000.0  # tonnes to ktonnes

sew_df

,sew_n,sew_p,sew_po4,sew_no3,sew_nh4,sew_Hg,sew_S.P.M.
ospar_region,,,,,,,
NORWAY,13.899220,1.160883,0.696530,0.694961,10.424415,0.0,0.0
LOFOTEN-BARENTS SEA,1.602106,0.196986,0.118191,0.080105,1.201579,0.0,0.0
NORTH SEA,3.347287,0.316116,0.189670,0.167364,2.510465,0.0,0.0
NORWEGIAN SEA2,3.490155,0.419726,0.251835,0.174508,2.617616,0.0,0.0
SKAGERAK,5.459672,0.228056,0.136833,0.272984,4.094754,0.0,0.0


In [10]:
# Update sheet 5a
pars = [
    "Ni",
    "Pb",
    "NH4-N",
    "Total Cr",
    "NO3-N",
    "Zn",
    "As",
    "Cd",
    "P-Total",
    "SPM",
    "PO4-P",
    "N-Total",
    "Hg",
    "Cu",
]

update_spreadsheet_point_sources(temp_path, "5a", pars, "sew", sew_df)

### 2.2. Sheet 5b: Industrial effluents

In [11]:
# Get industrial data
cols = [i for i in umon_df.columns if i.split("_")[0] == "ind"]
ind_df = umon_df[cols].copy()

# Convert units
ind_df["ind_Hg"] = ind_df["ind_Hg"] / 1000.0  # kg to tonnes
ind_df["ind_nh4"] = ind_df["ind_nh4"] / 1000.0  # tonnes to ktonnes
ind_df["ind_no3"] = ind_df["ind_no3"] / 1000.0  # tonnes to ktonnes
ind_df["ind_n"] = ind_df["ind_n"] / 1000.0  # tonnes to ktonnes
ind_df["ind_po4"] = ind_df["ind_po4"] / 1000.0  # tonnes to ktonnes
ind_df["ind_p"] = ind_df["ind_p"] / 1000.0  # tonnes to ktonnes
ind_df["ind_S.P.M."] = ind_df["ind_S.P.M."] / 1000.0  # tonnes to ktonnes

ind_df

,ind_n,ind_p,ind_po4,ind_no3,ind_nh4,ind_S.P.M.,ind_TOC,ind_As,ind_Pb,ind_Cd,ind_Cu,ind_Zn,ind_Ni,ind_Cr,ind_Hg
ospar_region,,,,,,,,,,,,,,,
NORWAY,3.263164,0.193940,0.116364,0.163158,2.447373,746.614540,265.834,0.625772,4.049955,0.948291,8.945741,45.357130,9.273851,0.112390,2.084341e-02
LOFOTEN-BARENTS SEA,0.001134,0.000069,0.000041,0.000057,0.000851,410.959612,0.490,0.493000,0.000012,0.000018,0.008400,0.000000,0.558000,0.000000,4.120000e-07
NORTH SEA,0.398860,0.047714,0.028629,0.019943,0.299145,11.911256,185.780,0.001601,3.665245,0.890868,0.550599,43.518135,5.486144,0.103704,7.120610e-03
NORWEGIAN SEA2,1.460610,0.087484,0.052491,0.073030,1.095457,321.745788,61.024,0.001057,0.238575,0.051704,0.331736,0.481636,0.066284,0.000000,4.067288e-03
SKAGERAK,1.402560,0.058673,0.035204,0.070128,1.051920,1.997884,18.540,0.130114,0.146122,0.005701,8.055006,1.357359,3.163423,0.008686,9.655100e-03


In [12]:
# Update sheet 5b
pars = [
    "Ni",
    "Pb",
    "NH4-N",
    "Total Cr",
    "NO3-N",
    "Zn",
    "As",
    "Cd",
    "P-Total",
    "SPM",
    "PO4-P",
    "N-Total",
    "Hg",
    "Cu",
    "TOC",
]

update_spreadsheet_point_sources(temp_path, "5b", pars, "ind", ind_df)

### 2.3. Sheet 5c: Aquaculture discharges

In [13]:
# Get fish data
cols = [i for i in umon_df.columns if i.split("_")[0] == "fish"]
fish_df = umon_df[cols].copy()

# Convert units
fish_df["fish_nh4"] = fish_df["fish_nh4"] / 1000.0  # tonnes to ktonnes
fish_df["fish_no3"] = fish_df["fish_no3"] / 1000.0  # tonnes to ktonnes
fish_df["fish_n"] = fish_df["fish_n"] / 1000.0  # tonnes to ktonnes
fish_df["fish_po4"] = fish_df["fish_po4"] / 1000.0  # tonnes to ktonnes
fish_df["fish_p"] = fish_df["fish_p"] / 1000.0  # tonnes to ktonnes

fish_df

,fish_n,fish_p,fish_po4,fish_no3,fish_nh4,fish_Cu
ospar_region,,,,,,
NORWAY,17.063347,3.586710,2.474830,1.876968,13.650677,308.537697
LOFOTEN-BARENTS SEA,2.524172,0.532105,0.367152,0.277659,2.019338,45.611788
NORTH SEA,6.077617,1.282109,0.884655,0.668538,4.862094,109.825412
NORWEGIAN SEA2,8.430804,1.766033,1.218563,0.927388,6.744643,152.544772
SKAGERAK,0.030754,0.006463,0.004459,0.003383,0.024603,0.555725


In [14]:
# Update sheet 5c
pars = ["NH4-N", "NO3-N", "P-Total", "PO4-P", "N-Total", "Cu"]

update_spreadsheet_point_sources(temp_path, "5c", pars, "fish", fish_df)

### 2.4. Sheet 5d: Other discharges

This sheet is left blank

### 2.5. Sheet 5e: Total direct discharges

The sum of sewage, industrial and fish-farm discharges.

In [15]:
# Combine sew, ind and fish, then aggregate
for df in [sew_df, ind_df, fish_df]:
    df.reset_index(inplace=True)
    df.columns = [i.split("_")[1] for i in df.columns]

td_df = pd.concat([sew_df, ind_df, fish_df], axis=0, sort=True)
td_df = td_df.groupby("region").sum()

td_df.columns = ["td_" + i for i in td_df.columns]

td_df

,td_As,td_Cd,td_Cr,td_Cu,td_Hg,td_Ni,td_Pb,td_S.P.M.,td_TOC,td_Zn,td_n,td_nh4,td_no3,td_p,td_po4
region,,,,,,,,,,,,,,,
LOFOTEN-BARENTS SEA,0.493000,0.000018,0.000000,45.620188,4.120000e-07,0.558000,0.000012,410.959612,0.490,0.000000,4.127412,3.221767,0.357821,0.729160,0.485385
NORTH SEA,0.001601,0.890868,0.103704,110.376012,7.120610e-03,5.486144,3.665245,11.911256,185.780,43.518135,9.823765,7.671704,0.855845,1.645939,1.102953
NORWAY,0.625772,0.948291,0.112390,317.483437,2.084341e-02,9.273851,4.049955,746.614540,265.834,45.357130,34.225731,26.522465,2.735087,4.941533,3.287724
NORWEGIAN SEA2,0.001057,0.051704,0.000000,152.876507,4.067288e-03,0.066284,0.238575,321.745788,61.024,0.481636,13.381568,10.457716,1.174927,2.273243,1.522888
SKAGERAK,0.130114,0.005701,0.008686,8.610730,9.655100e-03,3.163423,0.146122,1.997884,18.540,1.357359,6.892986,5.171277,0.346495,0.293192,0.176497


In [16]:
# Update sheet 5e
pars = [
    "Ni",
    "Pb",
    "NH4-N",
    "Total Cr",
    "NO3-N",
    "Zn",
    "As",
    "Cd",
    "P-Total",
    "SPM",
    "PO4-P",
    "N-Total",
    "Hg",
    "Cu",
    "TOC",
]

update_spreadsheet_point_sources(temp_path, "5e", pars, "td", td_df)

### 2.6. Sheet 6a: Monitored rivers

**Does "Inner Oslofjord" in the template correspond to "Alna"?** If so, I can fill-in one additional row in this table.

In [17]:
mon_df.reset_index(inplace=True)
tot_df = mon_df.groupby("ospar_region").sum()
trib_df = mon_df[mon_df["old_rid_group"] != "rid_11"].groupby("ospar_region").sum()

In [18]:
def update_spreadsheet_monitored_rivers(xlsx, sheet, pars, df_dict):
    """Update the OSPAR template for monitored rivers.

    Args:
        xslx:    Str. Path to Excel template
        sheet:   Str. Sheet name to update
        pars:    List. Parameter names in template to fill-in
        df_dict: Dict. {'tot':tot_df, 'trib':trib_df, 'main':rid11_df}
                 Values to fill-in

    Returns:
        None. The template is updated and saved.
    """
    import pandas as pd
    from openpyxl import load_workbook

    # Map Excel headings to df cols
    par_dict = {
        "SPM": "SPM",
        "TOC": "TOC",
        "PO4-P": "PO4-P",
        "P-Total": "TOTP",
        "NO3-N": "NO3-N",
        "NH4-N": "NH4-N",
        "N-Total": "TOTN",
        "As": "As",
        "Pb": "Pb",
        "Cd": "Cd",
        "Cu": "Cu",
        "Zn": "Zn",
        "Ni": "Ni",
        "Total Cr": "Cr",
        "Hg": "Hg",
    }

    # Map template names to df names and rows
    names_dict = {
        "Norwegian Sea (NO)": ("NORWEGIAN SEA2", "tot"),
        "Barents Sea (NO)": ("LOFOTEN-BARENTS SEA", "tot"),
        "Skagerrak (NO)": ("SKAGERAK", "tot"),
        "North Sea (NO)": ("NORTH SEA", "tot"),
        "Norway Total": ("NORWAY", "tot"),
        "Tributary Rivers - Norwegian Sea": ("NORWEGIAN SEA2", "trib"),
        "Tributary Rivers - Barents Sea": ("LOFOTEN-BARENTS SEA", "trib"),
        "Tributary Rivers - Skagerak": ("SKAGERAK", "trib"),
        "Tributary Rivers - North Sea": ("NORTH SEA", "trib"),
        "Orkla": (29778, "main"),
        "Vefsna": (29782, "main"),
        "Alta": (29779, "main"),
        "Glomma": (29617, "main"),
        "Drammenselva": (29612, "main"),
        u"Numedalslågen": (29615, "main"),
        "Skienselva": (29613, "main"),
        "Otra": (29614, "main"),
        "Orreelva": (29783, "main"),
        "Vosso": (29821, "main"),
    }

    # Open new file and get sheet
    wb = load_workbook(filename=xlsx)
    ws = wb[sheet]

    # Get row numbers
    row_dict = {}
    for item in ws["B12" : "B%s" % ws.max_row]:
        # Get cell properties
        cell = item[0]
        name = cell.value
        row = cell.row
        row_dict[name] = row

    # Get col numbers
    col_dict = {}
    for cell in ws["E9":"AK9"][0]:
        # Get cell properties
        par = cell.value
        col = cell.column
        col_dict[par] = col

    # Update spreadsheet
    for reg in names_dict.keys():
        df_idx, df_name = names_dict[reg]

        # Get df
        df = df_dict[df_name]

        for par in pars:
            # Get value from df
            val = df.loc[df_idx, par_dict[par]]

            # Get cell co-ords
            row = row_dict[reg] + 2
            col = col_dict[par]

            # Write value
            ws.cell(row=row, column=col).value = val

    # Save
    wb.save(xlsx)

In [19]:
# Update sheet 6a
pars = [
    "Ni",
    "Pb",
    "NH4-N",
    "Total Cr",
    "NO3-N",
    "Zn",
    "As",
    "Cd",
    "P-Total",
    "SPM",
    "PO4-P",
    "N-Total",
    "Hg",
    "Cu",
    "TOC",
]
df_dict = {"tot": tot_df, "main": rid11_df, "trib": trib_df}

update_spreadsheet_monitored_rivers(temp_path, "6a", pars, df_dict)

### 2.7. Sheet 6b: Unmonitored areas

In [20]:
# Get diff data
cols = [i for i in umon_df.columns if i.split("_")[0] == "diff"]
diff_df = umon_df[cols].copy()

# Convert units
diff_df["diff_nh4"] = diff_df["diff_nh4"] / 1000.0  # tonnes to ktonnes
diff_df["diff_no3"] = diff_df["diff_no3"] / 1000.0  # tonnes to ktonnes
diff_df["diff_n"] = diff_df["diff_n"] / 1000.0  # tonnes to ktonnes
diff_df["diff_po4"] = diff_df["diff_po4"] / 1000.0  # tonnes to ktonnes
diff_df["diff_p"] = diff_df["diff_p"] / 1000.0  # tonnes to ktonnes

diff_df

,diff_n,diff_p,diff_po4,diff_no3,diff_nh4
ospar_region,,,,,
NORWAY,32.172171,0.649499,0.159777,20.107607,1.769469
LOFOTEN-BARENTS SEA,6.009213,0.109707,0.026988,3.755758,0.330507
NORTH SEA,11.473191,0.188912,0.046472,7.170745,0.631026
NORWEGIAN SEA2,11.798066,0.257564,0.063361,7.373791,0.648894
SKAGERAK,2.891700,0.093315,0.022956,1.807312,0.159043


In [21]:
# Update sheet 6b
pars = ["NH4-N", "NO3-N", "P-Total", "PO4-P", "N-Total"]

update_spreadsheet_point_sources(temp_path, "6b", pars, "diff", diff_df)

### 2.8. Sheet 6c: Total inputs

**Note:** See e-mail from Csilla received 07/11/2017 at 13.43. This table should **not** include "point" discharges (`td_df`) - it's just the sum of tables 6a and 6b.

In [22]:
# Rename cols in diff_df
col_map = {
    "diff_n": "TOTN",
    "diff_p": "TOTP",
    "diff_po4": "PO4-P",
    "diff_no3": "NO3-N",
    "diff_nh4": "NH4-N",
}
diff_df.columns = [col_map[i] for i in diff_df.columns]

# Add to total_df
for col in diff_df.columns:
    tot_df[col] = tot_df[col] + diff_df[col]

# Update sheet 6c
pars = [
    "Ni",
    "Pb",
    "NH4-N",
    "Total Cr",
    "NO3-N",
    "Zn",
    "As",
    "Cd",
    "P-Total",
    "SPM",
    "PO4-P",
    "N-Total",
    "Hg",
    "Cu",
    "TOC",
]

df_dict = {"tot": tot_df, "main": rid11_df, "trib": trib_df}

update_spreadsheet_monitored_rivers(temp_path, "6c", pars, df_dict)

The following two cells are not longer used as I originally misunderstood what table 6c represents.

In [23]:
## Standardise col names
# diff_df.columns = [i.split('_')[1] for i in diff_df.columns]
# td_df.columns = [i.split('_')[1] for i in td_df.columns]
# td_df.index.name = 'ospar_region'
#
# col_dict = {'SPM':'S.P.M.',
#            'TOTN':'n',
#            'NH4-N':'nh4',
#            'NO3-N':'no3',
#            'TOTP':'p',
#            'PO4-P':'po4'}
# for col in col_dict.keys():
#    new_col = col_dict[col]
#    tot_df[new_col] = tot_df[col]
#    del tot_df[col]
#
## Reset index
# diff_df.reset_index(inplace=True)
# tot_df.reset_index(inplace=True)
# td_df.reset_index(inplace=True)
#
## Concat and aggregate
##tot_df = pd.concat([diff_df, td_df, tot_df], axis=0).groupby('ospar_region').sum() # See comment above
# tot_df = pd.concat([diff_df, tot_df], axis=0).groupby('ospar_region').sum()
#
## Rename cols
# tot_df.columns = ['tot_'+i for i in tot_df.columns]
#
# tot_df

In [24]:
## Update sheet 6c
# pars = ['Ni', 'Pb', 'NH4-N', 'Total Cr', 'NO3-N',
#        'Zn', 'As', 'Cd', 'P-Total', 'SPM', 'PO4-P',
#        'N-Total', 'Hg', 'Cu', 'TOC']
#
# update_spreadsheet_point_sources(temp_path, '6c', pars, 'tot', tot_df)

### 2.9. Sheet 7: Concentrations

In [25]:
# Read data
in_csv = f"../../../Results/Loads_CSVs/concs_and_flows_rid_11_{year}.csv"
conc_df = pd.read_csv(in_csv, index_col=0, encoding="utf-8")

cols = [
    "Hg_ng/l",
    "NH4-N_µg/l N",
    "NO3-N_µg/l N",
    "TOTN_µg/l N",
    "TOTP_µg/l P",
    "PO4-P_µg/l P",
    "TOC_mg C/l",
]
for col in cols:
    if col not in conc_df.columns:
        par = col.split("_")[0]
        conc_df[col] = np.nan
        conc_df[f"{par}_flag"] = np.nan

# Convert units
conc_df["Hg_ng/l"] = conc_df["Hg_ng/l"] / 1000  # ng to ug
conc_df["NH4-N_µg/l N"] = conc_df["NH4-N_µg/l N"] / 1000  # ug to mg
conc_df["NO3-N_µg/l N"] = conc_df["NO3-N_µg/l N"] / 1000  # ug to mg
conc_df["TOTN_µg/l N"] = conc_df["TOTN_µg/l N"] / 1000  # ug to mg
conc_df["TOTP_µg/l P"] = conc_df["TOTP_µg/l P"] / 1000  # ug to mg
conc_df["PO4-P_µg/l P"] = conc_df["PO4-P_µg/l P"] / 1000  # ug to mg
conc_df["TOC_mg C/l"] = conc_df["TOC_mg C/l"] * 1000  # mg to ug

# Get flags
cols = [i for i in conc_df.columns if i.split("_")[1] == "flag"]
lod_df = conc_df[cols]
lod_df.columns = [i.split("_")[0] for i in lod_df.columns]

# Get vals
cols = [
    i
    for i in conc_df.columns
    if ((i.split("_")[0] in lod_df.columns) and (i.split("_")[1] != "flag"))
]
conc_df = conc_df[cols]
conc_df.columns = [i.split("_")[0] for i in conc_df.columns]

# Rename
col_dict = {
    "SPM": "S.P.M.",
    "TOTN": "n",
    "NH4-N": "nh4",
    "NO3-N": "no3",
    "TOTP": "p",
    "PO4-P": "po4",
}
for col in col_dict.keys():
    new_col = col_dict[col]
    lod_df[new_col] = lod_df[col]
    conc_df[new_col] = conc_df[col]
    del conc_df[col], lod_df[col]

# Map Excel headings to df cols
par_dict = {
    "SPM": "S.P.M.",
    "TOC": "TOC",
    "PO4-P": "po4",
    "P-Total": "p",
    "NO3-N": "no3",
    "NH4-N": "nh4",
    "N-Total": "n",
    "As": "As",
    "Pb": "Pb",
    "Cd": "Cd",
    "Cu": "Cu",
    "Zn": "Zn",
    "Ni": "Ni",
    "Total Cr": "Cr",
    "Hg": "Hg",
}

# Map names to stns
names_dict = {
    "Orkla": 29778,
    "Vefsna": 29782,
    "Alta": 29779,
    "Glomma": 29617,
    "Drammenselva": 29612,
    "Numedalslågen": 29615,
    "Skienselva": 29613,
    "Otra": 29614,
    "Orreelva": 29783,
    "Vosso": 29821,
}

# Open new file and get sheet
wb = load_workbook(filename=temp_path)
ws = wb["7"]

# Get row numbers
row_dict = {}
for item in ws["B12" : "B%s" % ws.max_row]:
    # Get cell properties
    cell = item[0]
    name = cell.value
    row = cell.row
    row_dict[name] = row

# Get col numbers
col_dict = {}
for cell in ws["E9":"AK9"][0]:
    # Get cell properties
    par = cell.value
    col = cell.column
    col_dict[par] = col

pars = [
    "Ni",
    "Pb",
    "NH4-N",
    "Total Cr",
    "NO3-N",
    "Zn",
    "As",
    "Cd",
    "P-Total",
    "SPM",
    "PO4-P",
    "N-Total",
    "Hg",
    "Cu",
    "TOC",
]

# Update spreadsheet
for reg in names_dict.keys():
    for par in pars:
        if par_dict[par] not in conc_df.columns:
            conc_df[par_dict[par]] = np.nan
            lod_df[par_dict[par]] = np.nan

        # Get values from df
        # 1a. Lower average
        vals = conc_df.loc[names_dict[reg]].copy()[[par_dict[par]]].values
        lods = lod_df.loc[names_dict[reg]].copy()[[par_dict[par]]].fillna("0").values
        vals[(lods == "<")] = 0
        val = np.nanmean(vals)

        row = row_dict[reg]
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

        # 1b. Upper average
        val = conc_df.loc[names_dict[reg], par_dict[par]].mean()
        row = row_dict[reg]
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

        # 2. Min
        val = conc_df.loc[names_dict[reg], par_dict[par]].min()
        row = row_dict[reg] + 2
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

        # 3. Max
        val = conc_df.loc[names_dict[reg], par_dict[par]].max()
        row = row_dict[reg] + 3
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

        # 4. N
        n_samp = len(conc_df.loc[names_dict[reg]].copy()[[par_dict[par]]].dropna())
        if n_samp == 0:
            n_samp = np.nan
        row = row_dict[reg] + 5
        col = col_dict[par]
        ws.cell(row=row, column=col).value = n_samp

        # 5. N LOD
        n_lod = (~pd.isnull(lod_df.loc[names_dict[reg]].copy()[[par_dict[par]]])).sum()
        try:
            pct_lod = 100 * float(n_lod) / float(n_samp)
            if pct_lod < 30:
                val = "Yes"
            else:
                val = "No"
        except ZeroDivisionError:
            val = np.nan

        row = row_dict[reg] + 4
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

        # 6. Std
        val = conc_df.loc[names_dict[reg], par_dict[par]].std()
        row = row_dict[reg] + 7
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

# Save
wb.save(temp_path)

### 2.10. Sheet 9: Discharge

The notebook [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/recalculate_ospar_flows.ipynb) handles the OSPAR flow data. Run this, and then copy the results over to Sheet 9. 